In [1]:
%load_ext autoreload
%autoreload 2
import numpy as np
import sys
import pandas as pd
from pathlib import Path
from tqdm import tqdm
from collections import defaultdict, Counter
import ujson
from bootleg.run import run_model
sys.path.append("/dfs/scratch0/lorr1/projects/bootleg/tutorials")
from utils import load_mentions

# set up logging
import sys
import logging
from importlib import reload
reload(logging)
# Changing logging level will impact mention extractor
logging.basicConfig(stream=sys.stdout, format='%(asctime)s %(message)s', level=logging.INFO)
# logger = logging.getLogger(__name__)

root_dir = Path("/dfs/scratch0/lorr1/projects/bootleg/tutorial_data/")
cand_map = root_dir / 'data/wiki_entity_data/entity_mappings/alias2qids_wiki_filt.json'

In [2]:
qid2title = ujson.load(open(root_dir / 'data/wiki_entity_data/entity_mappings/qid2title.json'))

In [3]:
device = 0

In [4]:
nq_sample_orig = Path('/dfs/scratch0/lorr1/projects/bootleg-data/data/benchmarks/aida_0928_nosep/filtered/test.jsonl')
nq_sample_bootleg = Path('/dfs/scratch0/lorr1/projects/bootleg-data/data/benchmarks/aida_0928_nosep/filtered/test_bootleg_men.jsonl')

# nq_sample_orig = Path('/dfs/scratch0/lorr1/projects/bootleg-emmental/tutorial_data/data/nq/test_natural_questions_50.jsonl')
# nq_sample_bootleg = Path('/dfs/scratch0/lorr1/projects/bootleg-emmental/tutorial_data/data/nq/test_natural_questions_50_bootleg.jsonl')

In [5]:
from bootleg.end2end.extract_mentions import extract_mentions
extract_mentions(in_filepath=nq_sample_orig, out_filepath=nq_sample_bootleg, cand_map_file=cand_map, verbose=True)

100%|██████████| 309/309 [00:20<00:00, 15.24it/s]


In [7]:
orig_mentions_df = load_mentions(nq_sample_orig)
bootleg_mentions_df = load_mentions(nq_sample_bootleg)

# join dataframes and sample
res = pd.merge(orig_mentions_df, bootleg_mentions_df, on=['sentence'], suffixes=['_hand', '_bootleg'])
display(res.sample(50))

,sentence,aliases_hand,spans_hand,aliases_bootleg,spans_bootleg
1613,"vail , Colorado 1996-12-07 . vail , Colorado 1996-12-07","[alias_1344_0, alias_1344_1, alias_1344_0, alias_1344_1]","[[0, 1], [2, 3], [5, 6], [7, 8]]","[vail colorado, vail colorado]","[[0, 3], [5, 8]]"
203,"cricket - lara endures another miserable day . All-rounder Greg Blewett steered his side to a comfortable victory with an unbeaten 57 in 90 balls to the delight of the 42,442 crowd .","[alias_1177_0, alias_1177_9]","[[2, 3], [9, 11]]","[lara, greg blewett, 90 balls]","[[2, 3], [9, 11], [23, 25]]"
2408,nba basketball - standings after friday 's games . charlotte at seattle,"[alias_1383_0, alias_1383_51, alias_1383_52]","[[0, 1], [9, 10], [11, 12]]","[nba basketball, friday, charlotte, seattle]","[[0, 2], [5, 6], [9, 10], [11, 12]]"
1507,rtrs - Australian mp John Langmore formally resigns . canberra 1996-12-06,"[alias_1331_0, alias_1331_1, alias_1331_2, alias_1331_3]","[[0, 1], [2, 3], [4, 6], [9, 10]]","[rtrs, john langmore, canberra]","[[0, 1], [4, 6], [9, 10]]"
1559,"tokyo 1996-12-06 . Japan 's Economic Planning Agency has not changed its view that the economy is gradually recovering , despite relatively weak gross domestic product figures released on Tuesday , epa Vice Minister Shimpei Nukaya told reporters on Friday .","[alias_1337_0, alias_1337_1]","[[0, 1], [3, 4]]","[tokyo, japan, economic planning, gross domestic product, vice minister, friday]","[[0, 1], [3, 4], [5, 7], [23, 26], [32, 34], [39, 40]]"
883,"Multinational commander going back to east Zaire . If he decided it was necessary and safe for the aircrew , he would not hesitate to order airdrops of food for the refugees , even against the wishes of the government in Kinshasa and the Zairean rebels who control much of eastern Zaire , he said .","[alias_1241_0, alias_1241_9, alias_1241_10, alias_1241_11]","[[6, 7], [41, 42], [44, 45], [51, 52]]","[airdrops, kinshasa, zaire]","[[26, 27], [41, 42], [51, 52]]"
1928,"chonju , South Korea 1996-12-07 . Horii Manabu ( Japan ) 37.23 seconds ; 2 .","[alias_1360_0, alias_1360_2]","[[2, 4], [9, 10]]","[chonju, south korea, japan]","[[0, 1], [2, 4], [9, 10]]"
918,"Ex-minister , son killed in Central Africa unrest . bangui 1996-12-06","[alias_1245_0, alias_1245_1]","[[5, 7], [9, 10]]","[central africa, bangui]","[[5, 7], [9, 10]]"
1027,Plastic surgery gets boost in Brazil . Plastic surgery scares like the case in which Brazilian model Claudia Liz fell into a coma after being anaesthetised for a liposuction in October are not much of a deterrent .,"[alias_1259_0, alias_1259_12]","[[5, 6], [15, 16]]","[plastic surgery, brazil, plastic surgery, scares, coma, liposuction, deterrent]","[[0, 2], [5, 6], [7, 9], [9, 10], [22, 23], [28, 29], [36, 37]]"
2200,soccer - scottish league and cup results . soccer - scottish league and cup results .,"[alias_1372_0, alias_1372_0]","[[2, 3], [10, 11]]","[soccer, scottish league, soccer, scottish league]","[[0, 1], [2, 4], [8, 9], [10, 12]]"


In [14]:
BENCHMARK = "kore"
if BENCHMARK == "kore":
    data_dir = "/dfs/scratch0/lorr1/projects/bootleg-data/data/benchmarks/kore50_0114/filtered"
    entity_dir = "/dfs/scratch0/lorr1/projects/bootleg-data/data/benchmarks/kore50_0114/filtered/entity_db"
    alias_map = "alias2qids.json"
    test_file = "test.jsonl"
    raw_test_file = Path(data_dir) / test_file
elif BENCHMARK == "rss":
    data_dir = "/dfs/scratch0/lorr1/projects/bootleg-data/data/benchmarks/rss500wiki_0114/filtered"
    entity_dir = "/dfs/scratch0/lorr1/projects/bootleg-data/data/benchmarks/rss500wiki_0114/filtered/entity_db"
    alias_map = "alias2qids.json"
    test_file = "test_rss500.jsonl"
    raw_test_file = Path(data_dir) / test_file
elif BENCHMARK == "aida":
    data_dir = "/dfs/scratch0/lorr1/projects/bootleg-data/data/benchmarks/aida_0114/filtered"
    entity_dir = "/dfs/scratch0/lorr1/projects/bootleg-data/data/benchmarks/aida_0114/filtered/entity_db"
    alias_map = "alias2qids.json"
    test_file = "test.jsonl"
    raw_test_file = Path(data_dir) / test_file
else:
    data_dir = "/dfs/scratch0/lorr1/projects/bootleg-data/data/benchmarks/aida_0928_nosep/filtered"
    entity_dir = root_dir / 'data/wiki_entity_data'
    alias_map = "alias2qids_wiki_filt.json"
    test_file = "test_bootleg_men.jsonl"
    raw_test_file = Path(data_dir) / test_file

In [15]:
from bootleg.utils.parser.parser_utils import parse_boot_and_emm_args
from bootleg.utils.utils import dump_yaml_file, load_yaml_file

config_in_path = root_dir / 'models/bootleg_cased_mini/bootleg_config.yaml'

config_args = load_yaml_file(config_in_path)

# decrease number of data threads as this is a small file
config_args["run_config"]["dataset_threads"] = 2
config_args["run_config"]["log_level"] = "info"
# set the model checkpoint path 
config_args["emmental"]["model_path"] = str(root_dir / 'models/bootleg_cased_mini/bootleg_wiki.pth')

# set the path for the entity db and candidate map
config_args["data_config"]["entity_dir"] = entity_dir
config_args["data_config"]["alias_cand_map"] = alias_map

# set the data path and kore50 test file 
config_args["data_config"]["data_dir"] = data_dir

# to speed things up for the tutorial, we have already prepped the data with the mentions detected by Bootleg
config_args["data_config"]["test_dataset"]["file"] = test_file

# set the embedding paths 
config_args["data_config"]["emb_dir"] =  str(root_dir / 'data/emb_data')
config_args["data_config"]["word_embedding"]["cache_dir"] =  str(root_dir / 'data/emb_data/pretrained_bert_models')

# set the devie if on CPU
config_args["emmental"]["device"] = device

# save the new args (helps if you want to run things via command line)
# config_out_path = root_dir / f'models/bootleg_wiki/bootleg_wiki_config_eval_{BENCHMARK}.yaml'
# dump_yaml_file(config_out_path, config_args)
config_args = parse_boot_and_emm_args(config_args) # or you can pass in the config_out_path

In [ ]:
scores = run_model(mode="eval", config=config_args)
print(scores)

/dfs/scratch0/lorr1/projects/bootleg/bootleg/datasets/dataset.py:913: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  guid_dtype = np.dtype(
/dfs/scratch0/lorr1/env_bootleg_38/lib/python3.8/site-packages/numpy/core/memmap.py:230: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  descr = dtypedescr(dtype)
Building sent idx to row idx mapping: 100%|██████████| 50/50 [00:00<00:00, 6831.78it/s]


In [10]:
# parse the args for running
bootleg_label_file, bootleg_emb_file = run_model(mode="dump_preds", config=config_args)

/dfs/scratch0/lorr1/projects/bootleg/bootleg/datasets/dataset.py:913: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  guid_dtype = np.dtype(
Iterating over aliases: 100%|██████████| 15202497/15202497 [03:44<00:00, 67597.03it/s]
Reading in /dfs/scratch0/lorr1/projects/bootleg-data/data/benchmarks/aida_0928_nosep/filtered/prep/prep_test_dataset_files/create_examples_input/out_1.jsonl: 100%|██████████| 1232/1232 [00:03<00:00, 395.33it/s]
Reading in /dfs/scratch0/lorr1/projects/bootleg-data/data/benchmarks/aida_0928_nosep/filtered/prep/prep_test_dataset_files/create_examples_input/out_0.jsonl: 100%|██████████| 1233/1233 [00:03<00:00, 390.64it/s]
/dfs/scratch0/lorr1/env_bootleg_38/lib/python3.8/site-packages/numpy/core/memmap.py:230: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,

In [10]:
from utils import score_predictions, load_title_map, load_cand_map

bootleg_errors = score_predictions(raw_test_file, bootleg_label_file,
                                   qid2title,
                                   cands_map=load_cand_map(entity_dir, "alias2qids.json"),
                                   type_symbols=None, kg_symbols=None)

100%|██████████| 2465/2465 [00:00<00:00, 5308.70it/s]


In [11]:
print(bootleg_errors.shape)
print(bootleg_errors[bootleg_errors["pred_qid"] != bootleg_errors["gold_qid"]].shape)
print(bootleg_errors[bootleg_errors["pred_qid"] == bootleg_errors["gold_qid"]].shape)

(11360, 18)
(11360, 18)
(0, 18)


In [11]:
# Load prediction file
def load_predictions(file):
    lines = {}
    num_lines = sum(1 for line in open(file))
    with open(file) as f:
        for line in tqdm(f, total=num_lines):
            line = ujson.loads(line)
            lines[line["sent_idx_unq"]] = line
    return lines
    
# Reads in the files, joining on the sentence index
# If AIDA, we only keep the second sentence predictions as we merge the sentences together
def read_label_file(label_file, test_file, is_aida=False, threshold=0.0):
    num_lines = sum(1 for line in open(test_file))
    preds = load_predictions(label_file)
    gold_sentences = {}
    pred_sentences = {}
    with open(test_file) as f:
        for line in tqdm(f, total=num_lines):
            line = ujson.loads(line)
            offset_span = len(line["sentence"].split(" . ")[0].strip().split())+1
            sent_idx = line["sent_idx_unq"]
            gold_qids = line["qids"]
            gold_spans = line["spans"]
            gold_aliases = line["aliases"]
            gold_golds = line["gold"]
            if is_aida:
                filtered_values = list(filter(lambda x: x[3] is True, zip(gold_qids, gold_spans, gold_aliases, gold_golds)))
                if len(filtered_values) > 0:
                    gold_qids, gold_spans, gold_aliases, gold_golds = zip(*filtered_values)
                else:
                    gold_qids, gold_spans, gold_aliases, gold_golds = [], [], [], []
                    
            pred_aliases = preds.get(sent_idx, {}).get("aliases", [])
            pred_qids = preds.get(sent_idx, {}).get("qids", [])
            pred_spans = preds.get(sent_idx, {}).get("spans", [])
            pred_probs = preds.get(sent_idx, {}).get("probs", [])
            pred_cand_probs = preds.get(sent_idx, {}).get("cand_probs", [])
            pred_cands = preds.get(sent_idx, {}).get("cands", [])
            assert len(pred_aliases) == len(pred_qids) == len(pred_spans) == len(pred_probs) == len(pred_cand_probs) == len(pred_cands)
            filt_qids, filt_spans, filt_aliases, filt_probs, filt_cand_probs, filt_cands = [], [], [], [], [], []
            for q, s, a, p, pcp, pc in zip(pred_qids, pred_spans, pred_aliases, pred_probs, pred_cand_probs, pred_cands):
                if p > threshold:
                    # As we stitch the first sentence of the doc with the remaining sentences,
                    # we must undo that when scoring
                    if is_aida:
                        if s[0] >= offset_span:
                            filt_qids.append(q)
                            filt_spans.append(s)
                            filt_aliases.append(a)
                            filt_probs.append(p)
                            filt_cand_probs.append(pcp)
                            filt_cands.append(pc)
                    else:
                        filt_qids.append(q)
                        filt_spans.append(s)
                        filt_aliases.append(a)
                        filt_probs.append(p)
                        filt_cand_probs.append(pcp)
                        filt_cands.append(pc)
                        
            gold_sentences[sent_idx] = {
                "sentence": line["sentence"],
                "sent_idx": sent_idx,
                "doc_sent_idx": line.get("doc_sent_idx", line.get("doc_id", -1)),
                "aliases": gold_aliases,
                "qids": gold_qids,
                "spans": gold_spans
            }
            
            pred_sentences[sent_idx] = {
                "sentence": line["sentence"],
                "sent_idx": sent_idx,
                "doc_sent_idx": line.get("doc_sent_idx", line.get("doc_id", -1)),
                "aliases": filt_aliases,
                "qids": filt_qids,
                "spans": filt_spans,
                "probs": filt_probs,
                "cands": filt_cands,
                "cand_probs": filt_cand_probs
            }
                
    return gold_sentences, pred_sentences

def print_precision_and_recall(gold_sentences, pred_sentences):
    print("SENTENCES", len(gold_sentences))
    micro_crc, micro_gold, micro_pred = 0, 0, 0
    crc = defaultdict(int)
    for sent_idx in gold_sentences:
        gold_qids = gold_sentences[sent_idx]["qids"]
        pred_qids = pred_sentences[sent_idx]["qids"]
        num_intersection = len(list((Counter(gold_qids) & Counter(pred_qids)).elements()))
        crc[sent_idx] = num_intersection
        micro_crc += crc[sent_idx]
        micro_gold += len(gold_qids)
        micro_pred += len(pred_qids)
    macro_recall = np.mean([crc[sent_idx]/len(gold_sentences[sent_idx]["qids"]) for sent_idx in gold_sentences])
    macro_prec = np.mean([crc[sent_idx]/len(pred_sentences[sent_idx]["qids"])
                          if len(pred_sentences[sent_idx]["qids"]) > 0
                          else 1 for sent_idx in gold_sentences])
    print("MICRO")
    micro_prec = micro_crc/micro_pred
    micro_recall = micro_crc/micro_gold
    print("PREC", micro_prec, "RECALL", micro_recall, "F1", (2*micro_prec*micro_recall/(micro_prec + micro_recall)))
    print("MACRO")
    print("PREC", macro_prec, "RECALL", macro_recall, "F1", (2*macro_prec*macro_recall/(macro_prec + macro_recall)))
    
    
def span_overlap(a, b):
    return max(0, min(a[1], b[1]) - max(a[0], b[0]))

def create_error(gold_qid, gold_alias, gold_span, gold_label, pred_qid, pred_alias, pred_span, sent_obj, gold_aliases, gold_qids, gold_spans, found_aliases, found_spans, pred_qids, error_type):
    return {
        "error_type": error_type,
        "doc_idx": sent_obj["doc_sent_idx"],
        "sentence": sent_obj["sentence"],
        "gold_qid": gold_qid,
        "gold_title": qid2title.get(gold_qid, ""),
        "gold_alias": gold_alias,
        "gold_label": gold_label,
        "gold_span": gold_span,
        "pred_qid": pred_qid,
        "pred_title": qid2title.get(pred_qid, ""),
        "pred_alias": pred_alias,
        "pred_span": pred_span,
        "all_gold_aliases": gold_aliases,
        "all_gold_qids": gold_qids,
        "all_gold_spans": gold_spans,
        "all_pred_aliases": found_aliases,
        "all_pred_spans": found_spans,
        "all_pred_qids": pred_qids
    }

def compute_errors(gold_sentence, pred_sentence):
    # read in first file and map by index for fast retrieval
    total_mentions = 0
    correct_mentions = 0
    pred_mentions = 0
    errors = []
    gold_aliases = gold_sentence['aliases']
    gold_spans = gold_sentence['spans']
    gold_qids = gold_sentence['qids']
    is_gold_label = gold_sentence.get('gold', [True]*len(gold_aliases))

    pred_aliases = pred_sentence['aliases']
    pred_spans = pred_sentence['spans']
    pred_qids = pred_sentence['qids']

    total_mentions += len(gold_aliases)
    pred_mentions += len(pred_qids)

    for gold_alias, gold_qid, gold_span, gold_label in zip(gold_aliases, gold_qids, gold_spans, is_gold_label):
        gold_span_start, gold_span_end = gold_span
        overlap_scores = np.array([span_overlap(gold_span, ps) for ps in pred_spans])
        # print(gold_span, pred_spans, overlap_scores, gold_sentence['sentence'])
        if any([o > 0 for o in overlap_scores]):
            pred_idx = np.argmax(overlap_scores)
            if gold_qid == pred_qids[pred_idx]:
                correct_mentions += 1
                errors.append(create_error(gold_qid, gold_alias, gold_span, gold_label, pred_qids[pred_idx], pred_aliases[pred_idx], pred_spans[pred_idx], gold_sentence, gold_aliases,
                                                        gold_qids, gold_spans, pred_aliases, pred_spans,
                                                        pred_qids, error_type='correct'))
            else:
                errors.append(create_error(gold_qid, gold_alias, gold_span, gold_label, pred_qids[pred_idx], pred_aliases[pred_idx], pred_spans[pred_idx], gold_sentence, gold_aliases,
                                                        gold_qids, gold_spans, pred_aliases, pred_spans,
                                                        pred_qids, error_type='wrong_entity'))
        else:
            errors.append(create_error(gold_qid, gold_alias, gold_span, gold_label, "N/A", "N/A", "N/A", gold_sentence, gold_aliases, gold_qids,
                                                        gold_spans, pred_aliases, pred_spans,
                                                         pred_qids, error_type='missing_mention'))

    for pred_alias, pred_span, pred_qid in zip(pred_aliases, pred_spans, pred_qids):
        pred_span_start, pred_span_end = pred_span
        fuzzy_gold_left = [pred_span_start-1,pred_span_end]
        fuzzy_gold_right = [pred_span_start+1,pred_span_end]
        if pred_span not in gold_spans and fuzzy_gold_left not in gold_spans and fuzzy_gold_right not in gold_spans and pred_qid != 'NC':
            errors.append(create_error("N/A", "N/A", "N/A", "N/A", pred_qid, pred_alias, pred_span, gold_sentence, gold_aliases, gold_qids, gold_spans, pred_aliases, pred_spans,
                                                      pred_qids, error_type='extra_mention'))
    return errors

In [12]:
gold_sentences, pred_sentences = read_label_file(bootleg_label_file, nq_sample_orig, is_aida=True, threshold=0.0)

100%|██████████| 2465/2465 [00:00<00:00, 45792.39it/s]


In [13]:
# Generate errors
print_precision_and_recall(gold_sentences, pred_sentences)
all_errors = []
for sent_idx, gold_sent in gold_sentences.items():
    pred_sent = pred_sentences[sent_idx]
    all_errors.extend(compute_errors(gold_sent, pred_sent))
    
error_df = pd.DataFrame(all_errors)

SENTENCES 2465
MICRO
PREC 0.5469678953626635 RECALL 0.717948717948718 F1 0.6209024296182029
MACRO
PREC 0.6303627167145527 RECALL 0.7115612237088426 F1 0.6685053490353806


In [16]:
error_df[error_df["error_type"] == "extra_mention"].sample(20)

In [73]:
import string, spacy, nltk, re
from bootleg.end2end.extract_mentions import get_new_to_old_dict, get_lnrm
nlp = spacy.load("en_core_web_sm", disable=["parser", "ner"])
ALL_STOPWORDS = nlp.Defaults.stop_words
PUNC = string.punctuation
KEEP_POS = {"PROPN", "NOUN"} # ADJ, VERB, ADV, SYM
PLURAL = {"s", "'s"}
table = str.maketrans(
    dict.fromkeys(PUNC)
)  # OR {key: None for key in string.punctuation}


def find_aliases_in_sentence_new(sentence, all_aliases, max_alias_len=6):
    used_aliases = []
    # Remove multiple spaces and replace with single - tokenization eats multiple spaces but ngrams doesn't which can cause parse issues
    sentence = re.sub(" +", " ", sentence)

    doc = nlp(sentence)
    split_sent = sentence.split()
    new_to_old_span = get_new_to_old_dict(split_sent)
    # find largest aliases first
    for n in range(max_alias_len + 1, 0, -1):
        grams = nltk.ngrams(doc, n)
        j_st = -1
        j_end = n - 1
        for gram_words in grams:
            j_st += 1
            j_end += 1
            j_st_adjusted = new_to_old_span[j_st]
            j_end_adjusted = new_to_old_span[j_end]
            # Check if nlp has split the word and we are looking at a subword mention - which we don't want
            is_subword = j_st_adjusted == j_end_adjusted
            if j_st > 0:
                is_subword = is_subword | (j_st_adjusted == new_to_old_span[j_st - 1])
            # j_end is exclusive and should be a new word from the previous j_end-1
            is_subword = is_subword | (j_end_adjusted == new_to_old_span[j_end - 1])
            if is_subword:
                continue
            # Assert we are a full word
            assert j_st_adjusted != j_end_adjusted, f"Something went wrong getting mentions for {sentence}"
            # If single word and not in a POS we care about, skip
            if len(gram_words) == 1 and gram_words[0].pos_ not in KEEP_POS:
                continue
            # If multiple word and not any word in a POS we care about, skip
            if len(gram_words) > 1 and not any(g.pos_ in KEEP_POS for g in gram_words):
                continue
            # print("@", gram_words, [g.pos_ for g in gram_words])
            # If we are part of a proper noun, make sure there isn't another part of the proper noun to the 
            # left or right - this means we didn't have the entire name in our alias and we should skip
            if len(gram_words) == 1 and gram_words[0].pos_ == "PROPN":
                if j_st > 0 and doc[j_st - 1].pos_ == "PROPN":
                    continue
                # End spans are exclusive so no +1
                if j_end < len(doc) and doc[j_end].pos_ == "PROPN":
                    continue
            # print("3", j_st, gram_words, [g.pos_ for g in gram_words])
            # We don't want punctuation words to be used at the beginning/end unless it's capitalized
            # or first word of sentence
            if (
                gram_words[-1].text in PLURAL
                or gram_words[0].text in PLURAL
                or (
                    gram_words[0].text.lower() in ALL_STOPWORDS
                    and (not gram_words[0].text[0].isupper() or j_st == 0)
                )
            ):
                continue
            # If the word starts with punctuation and there is a space in between, also continue; keep
            # if punctuation is part of the word boundary
            # print("4", j_st, gram_words, [g.pos_ for g in gram_words])
            if (
                (
                    gram_words[0].text in PUNC
                    and (j_st+1 >= len(doc) or new_to_old_span[j_st] != new_to_old_span[j_st+1])
                )
                or
                (
                    gram_words[-1].text in PUNC
                    and (j_end-2 < 0 or new_to_old_span[j_end-1] != new_to_old_span[j_end-2])
                )
            ):
                continue
            joined_gram = " ".join(split_sent[j_st_adjusted:j_end_adjusted])
            # If 's in alias, make sure we remove the space and try that alias, too
            joined_gram_merged_plural = joined_gram.replace(" 's", "'s")
            # If PUNC in alias, make sure we remove the space and try that alias, too
            joined_gram_merged_nopunc = joined_gram_merged_plural.translate(table)
            gram_attempt = get_lnrm(joined_gram, strip=True, lower=True)
            gram_attempt_merged_plural = get_lnrm(
                joined_gram_merged_plural, strip=True, lower=True
            )
            gram_attempt_merged_nopunc = get_lnrm(
                joined_gram_merged_nopunc, strip=True, lower=True
            )
            # Remove numbers
            if (
                gram_attempt.isnumeric()
                or joined_gram_merged_plural.isnumeric()
                or gram_attempt_merged_nopunc.isnumeric()
            ):
                continue
            final_gram = None
            # print("4", gram_attempt, [g.pos_ for g in gram_words])
            if gram_attempt in all_aliases:
                final_gram = gram_attempt
            elif gram_attempt_merged_plural in all_aliases:
                final_gram = gram_attempt_merged_plural
            elif gram_attempt_merged_nopunc in all_aliases:
                final_gram = gram_attempt_merged_nopunc
                # print("5", final_gram, [g.pos_ for g in gram_words])
            # print("FINAL GRAM", final_gram)
            if final_gram is not None:
                keep = True
                # We start from the largest n-grams and go down in size. This prevents us from adding an alias that is a subset of another.
                # For example: "Tell me about the mother on how I met you mother" will find "the mother" as alias and "mother". We want to
                # only take "the mother" and not "mother" as it's likely more descriptive of the real entity.
                for u_al in used_aliases:
                    u_j_st = u_al[1]
                    u_j_end = u_al[2]
                    if j_st_adjusted < u_j_end and j_end_adjusted > u_j_st:
                        keep = False
                        break
                if not keep:
                    continue
                used_aliases.append(tuple([final_gram, j_st_adjusted, j_end_adjusted]))
    # sort based on span order
    aliases_for_sorting = sorted(used_aliases, key=lambda elem: [elem[1], elem[2]])
    used_aliases = [a[0] for a in aliases_for_sorting]
    spans = [[a[1], a[2]] for a in aliases_for_sorting]
    assert all([sp[1] <= len(doc) for sp in spans]), f"{spans} {sentence}"
    print(spans)
    return used_aliases, spans

In [7]:
%load_ext autoreload
%autoreload 2
from bootleg.end2end.bootleg_annotator import BootlegAnnotator

ann = BootlegAnnotator(cache_dir='/dfs/scratch0/lorr1/projects/bootleg_cache', device=0)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


/dfs/scratch0/lorr1/projects/bootleg/bootleg/layers/alias_to_ent_encoder.py:97: UserWarning: The given NumPy array is not writeable, and PyTorch does not support non-writeable tensors. This means you can write to the underlying (supposedly non-writeable) NumPy array using the tensor. You may want to copy the array to protect its data or make it writeable before converting it to a tensor. This type of warning will be suppressed for the rest of this program. (Triggered internally at  /pytorch/torch/csrc/utils/tensor_numpy.cpp:141.)
  alias2entity_table = torch.from_numpy(alias2entity_table)


In [25]:
ann.set_threshold(0.0)
ann.verbose = False

In [74]:
ann.label_mentions(["i want to buy a avro lincoln"])["titles"]

[['Avro Lincoln']]

In [76]:
s = "nhl ice hockey - standings after friday 's games . philadelphia 15 12 2 81 78 32"
ann.label_mentions(s, label_func=find_aliases_in_sentence_new)["titles"]

[[6, 7], [10, 11]]


[['Robin Friday', 'Philadelphia Flyers']]